#This RAG framework for HaluBench dataset by using HybridSearch method through BGE-M3 Embedding model, and uses Ollama-Mistral model for generating answers through Llamaindex


In [ ]:
!pip install -q llama-index
!pip install -q llama-index-llms-huggingface
!pip install -q llama-index-embeddings-huggingface
!pip install -q torch sentence-transformers
!pip install -q -U pymilvus
!pip install -q datasets
!pip install -q transformers
!pip install -q spacy
!python -m spacy download en_core_web_sm
!pip install -q evaluate
!pip install -q tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.8/173.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.8/402.8 

In [ ]:
import pandas as pd
csv_path = "/content/halubench_short.csv"
data = pd.read_csv(csv_path)
required_columns = ['question', 'contexts', 'ground_truth']
if not all(col in data.columns for col in required_columns):
    raise ValueError(f"The dataset must contain these columns: {required_columns}")
# Extract 'contexts' and 'questions'
text_data = data['contexts'].tolist()
questions_data = data['question'].tolist()

In [ ]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import torch

def split_text_by_length(text, max_length=4096):
    """
    Split the input text into chunks of max_length, ensuring no chunk exceeds max_length.
    """
    split_texts = []
    for i in range(0, len(text), max_length):
        chunk = text[i:i + max_length]
        if len(chunk) <= max_length:
            split_texts.append(chunk)
    return split_texts

split_texts = []
for text in text_data:
    split_texts.extend(split_text_by_length(text, max_length=4096))
#Initialize a list to store the mapping between questions and their context chunks
mapped_questions = []

# Map each question to its corresponding context chunks
for idx, text in enumerate(text_data):
    chunks = split_text_by_length(text, max_length=4096)  # Split context into chunks
    for _ in chunks:  # For each chunk of context, repeat the question
        mapped_questions.append(questions_data[idx])  # Map the question to each chunk

# Now both mapped_questions and split_texts should have the same length
assert len(mapped_questions) == len(split_texts), "Mapped questions and split texts are not aligned."

# Display the split chunks
for i, chunk in enumerate(split_texts):
    print(f"Chunk {i+1}:")
    print(chunk)
    print()

Chunk 1:
Hoping to rebound from the road loss to the Chargers, the Rams went home for Week 9, as they fought the Kansas City Chiefs in a "Show Me State Showdown". The Chiefs struck first as RB Larry Johnson got a 1-yard TD run for the only score of the period.  In the second quarter, things got worse for the Rams as QB Damon Huard completed a 3-yard TD pass to TE Tony Gonzalez, while kicker Lawrence Tynes nailed a 42-yard field goal.  St. Louis got on the board with RB Steven Jackson getting a 2-yard TD run, yet Huard and Gonzalez hooked up with each other again on a 25-yard TD strike.  Rams kicker Jeff Wilkins made a 41-yard field goal to end the half.  In the third quarter, QB Marc Bulger completed a 2-yard TD pass to WR Kevin Curtis for the only score of the period, yet the only score of the fourth quarter came from Huard completing an 11-yard TD pass to TE Kris Wilson. With the loss, the Rams fell to 4-4.

Chunk 2:
As of the census of 2000, there were 218,590 people, 79,667 househo

#Retrieval

In [ ]:
from pymilvus import MilvusClient, utility

client = MilvusClient("milvus.db")

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: 2a7b45bfd5bd4981a893880c0bd0aadb


In [ ]:
from pymilvus import (
    connections, FieldSchema, CollectionSchema, DataType, Collection, utility
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import torch

# Connect to Milvus using the cluster information from Zilliz Cloud
connections.connect(
    alias="default",
    uri="https://in03-7a0bd7c7d07bbe2.serverless.gcp-us-west1.cloud.zilliz.com",
    token="2fd0d35c5daca9322a69814ef89023098b12df8a1e716334ac601149f386dbd4fd7b2ad5265df24b6a7452ca1ddd097c1fb572f8"
)

# Optionally, check the connection status
connected = connections.has_connection(alias="default")
print(f"Connection established: {connected}")

Connection established: True


In [ ]:
!pip install -q --upgrade pymilvus
!pip install -q "pymilvus[model]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.3 MB/s eta 0:00:00


In [ ]:
from milvus_model.hybrid import BGEM3EmbeddingFunction

# Initialize Embedding Model
# Using the BGE-M3 embedding model to generate both dense and sparse embeddings
ef = BGEM3EmbeddingFunction(use_fp16=False, device="cpu")
dense_dim = ef.dim["dense"]

start to install package: peft
successfully installed package: peft
start to install package: FlagEmbedding>=1.2.2
successfully installed package: FlagEmbedding>=1.2.2


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

...

sparse_linear.pt:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

model.onnx_data:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/FlagEmbedding/BGE_M3/modeling.py:335: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  colbert_state_dict = torch.load(os.path.join(mode

In [ ]:
# Generate embeddings for documents (contexts)
docs_embeddings = ef(split_texts)

In [ ]:
from datasets import load_dataset

from pymilvus import (
    AnnSearchRequest,
     WeightedRanker
)

In [ ]:
# Define Collection Schema
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="question", dtype=DataType.VARCHAR, max_length=512),  # Field for question
    FieldSchema(name="context", dtype=DataType.VARCHAR, max_length=8192),  # Field for context
    FieldSchema(name="dense_vector", dtype=DataType.FLOAT_VECTOR, dim=dense_dim),  # Dense vector field
    FieldSchema(name="sparse_vector", dtype=DataType.SPARSE_FLOAT_VECTOR)  # Sparse vector field
]
schema = CollectionSchema(fields)

In [ ]:
# Create or Load Collection
col_name = "rag_hybrid_search"
if utility.has_collection(col_name):
    Collection(col_name).drop()
col = Collection(col_name, schema, consistency_level="Strong")

In [ ]:
# Create Indices for both Sparse and Dense Vectors
sparse_index = {"index_type": "SPARSE_INVERTED_INDEX", "metric_type": "IP"}  # Sparse vector index
dense_index = {"index_type": "AUTOINDEX", "metric_type": "IP"}  # Dense vector index
col.create_index("sparse_vector", sparse_index)
col.create_index("dense_vector", dense_index)

Status(code=0, message=)

In [ ]:
# Load Collection into Memory for Efficient Search
col.load()

In [ ]:
# Insert Data into the Collection
# Batch insertions for efficiency
batch_size = 60
for i in range(0, len(split_texts), batch_size):
    batched_entities = [
        mapped_questions[i : i + batch_size],  # Insert questions, now properly aligned with context chunks
        split_texts[i : i + batch_size],  # Insert pre-split context chunks
        docs_embeddings["dense"][i : i + batch_size],  # Insert dense vector embeddings
        docs_embeddings["sparse"][i : i + batch_size],  # Insert sparse vector embeddings
    ]

    # Capture the response and print it
    response = col.insert(batched_entities)
    print(f"Inserted batch {i // batch_size + 1} with response: {response}")

# After insertion, refresh the collection statistics
col.flush()  # Ensure the data is flushed to the collection
print(f"Number of entities inserted: {col.num_entities}")

Inserted batch 1 with response: (insert count: 60, delete count: 0, upsert count: 0, timestamp: 453056070731169797, success count: 60, err count: 0, cost: 110)
Inserted batch 2 with response: (insert count: 32, delete count: 0, upsert count: 0, timestamp: 453056070914932738, success count: 32, err count: 0, cost: 81)
Number of entities inserted: 92


In [ ]:
# Initialize the list to store embeddings for the questions
question_embeddings = []

# Generate dense and sparse embeddings for each question using the BGEM3EmbeddingFunction
for question in questions_data:
    embeddings = ef([question])  # Use BGEM3EmbeddingFunction directly to generate embeddings
    question_embeddings.append(embeddings)

In [ ]:
# Function to perform dense vector search
def dense_search(col, query_dense_embedding, limit=10):
    search_params = {"metric_type": "IP", "params": {}}
    res = col.search(
        [query_dense_embedding],  # Directly pass the dense embedding as a list or array
        anns_field="dense_vector",
        limit=limit,
        output_fields=["context", "question"],  # Retrieve context and question fields
        param=search_params,
    )[0]
    return [(hit.get("question"), hit.get("context")) for hit in res]  # Return both question and context

In [ ]:
# Function to perform sparse vector search
def sparse_search(col, query_sparse_embedding, limit=10):
    search_params = {"metric_type": "IP", "params": {}}
    res = col.search(
        [query_sparse_embedding],  # Directly pass the sparse embedding as a list or array
        anns_field="sparse_vector",
        limit=limit,
        output_fields=["context", "question"],  # Retrieve context and question fields
        param=search_params,
    )[0]
    return [(hit.get("question"), hit.get("context")) for hit in res]

In [ ]:
# Hybrid search combining both dense and sparse embeddings
def hybrid_search(
    col,
    query_dense_embedding,
    query_sparse_embedding,
    sparse_weight=1.0,
    dense_weight=1.0,
    limit=10,
):
    dense_search_params = {"metric_type": "IP", "params": {}}
    dense_req = AnnSearchRequest(
        [query_dense_embedding], "dense_vector", dense_search_params, limit=limit
    )

    sparse_search_params = {"metric_type": "IP", "params": {}}
    sparse_req = AnnSearchRequest(
        [query_sparse_embedding], "sparse_vector", sparse_search_params, limit=limit
    )

    # Weight the two types of vectors in the ranking
    rerank = WeightedRanker(sparse_weight, dense_weight)

    # Perform the hybrid search
    res = col.hybrid_search(
        [sparse_req, dense_req], rerank=rerank, limit=limit, output_fields=["question", "context"]
    )[0]

    return [(hit.get("question"), hit.get("context")) for hit in res]  # Return both question and context

In [ ]:
# for embeddings in question_embeddings:
#     query_dense_embedding = embeddings['dense'][0].tolist()  # Use pre-generated dense embeddings
#     query_sparse_embedding = embeddings['sparse']  # Use pre-generated sparse embeddings

#     # Perform the hybrid search
#     results = hybrid_search(col, query_dense_embedding, query_sparse_embedding, limit=5)

#     # Display results
#     for i, (retrieved_question, context) in enumerate(results):
#         print(f"Result {i+1}:")
#         print(f"Retrieved Question: {retrieved_question}")
#         print(f"Context: {context}")
#         print('-' * 50)

Streaming output truncated to the last 5000 lines.
--------------------------------------------------
Result 4:
Retrieved Question: What is the FY2021 return on assets (ROA) for Apple? ROA is defined as: FY2021 net income / (average total assets between FY2020 and FY2021). Round your answer to two decimal places. Please base your judgments on the information provided primarily in the income statement and the statement of financial position.
Context: Apple Inc.
CONSOLIDATED STATEMENTS OF OPERATIONS
(In millions, except number of shares which are reflected in thousands and per share amounts)
Years ended
September 25, 
2021
September 26, 
2020
September 28, 
2019
Net sales:
 Products
$
297,392 
$
220,747 
$
213,883 
 Services
68,425 
53,768 
46,291 
Total net sales
365,817 
274,515 
260,174 
Cost of sales:
 Products
192,266 
151,286 
144,996 
 Services
20,715 
18,273 
16,786 
Total cost of sales
212,981 
169,559 
161,782 
Gross margin
152,836 
104,956 
98,392 
Operating expenses:
Research

In [ ]:
# Use pre-generated embeddings stored in `question_embeddings` for each question
for i, question in enumerate(questions_data):
    # Retrieve the pre-generated dense and sparse embeddings for the current question
    embeddings = question_embeddings[i]  # Use the stored embeddings for the current question

    # Extract dense embedding as a list of floats
    query_dense_embedding = embeddings['dense'][0].tolist()  # Convert NumPy array to list of floats

    # Use the sparse matrix as is
    query_sparse_embedding = embeddings['sparse']  # Use the sparse matrix in CSR format

    # Perform the hybrid search using both embeddings
    results = hybrid_search(col, query_dense_embedding, query_sparse_embedding, limit=5)

    # Display the results
    print(f"Results for question: {question}")
    for j, (retrieved_question, context) in enumerate(results):
        print(f"Result {j+1}:")
        print(f"Retrieved Question: {retrieved_question}")
        print(f"Context: {context}")
        print('-' * 50)

Streaming output truncated to the last 5000 lines.
151,286 
144,996 
 Services
20,715 
18,273 
16,786 
Total cost of sales
212,981 
169,559 
161,782 
Gross margin
152,836 
104,956 
98,392 
Operating expenses:
Research and development
21,914 
18,752 
16,217 
Selling, general and administrative
21,973 
19,916 
18,245 
Total operating expenses
43,887 
38,668 
34,462 
Operating income
108,949 
66,288 
63,930 
Other income/(expense), net
258 
803 
1,807 
Income before provision for income taxes
109,207 
67,091 
65,737 
Provision for income taxes
14,527 
9,680 
10,481 
Net income
$
94,680 
$
57,411 
$
55,256 
Earnings per share:
Basic
$
5.67 
$
3.31 
$
2.99 
Diluted
$
5.61 
$
3.28 
$
2.97 
Shares used in computing earnings per share:
Basic
16,701,272 
17,352,119 
18,471,336 
Diluted
16,864,919 
17,528,214 
18,595,651 
See accompanying Notes to Consolidated Financial Statements.
Apple Inc. | 2021 Form 10-K | 29
Apple Inc.
CONSOLIDATED BALANCE SHEETS
(In millions, except number of shares which

In [ ]:
for j, (retrieved_question, context) in enumerate(results):
    print(f"Hybrid Search Result {j+1}:")
    print(f"Retrieved Question: {retrieved_question}")
    print(f"Context: {context}")
    print('-' * 50)

Hybrid Search Result 1:
Retrieved Question: What has the increasing abundance of affordable, sensitive, high-throughput genome sequencing technologies  led to?
Context: in various tissues and their potential therapeutic applications. 6 These RBPs include tristetraprolin and AUF1, which promote degradation of AU-rich element (ARE)-containing mRNA; members of the Roquin and Regnase families, which respectively promote or effect degradation of mRNAs harbouring stem-loop structures; and the increasingly apparent role of the RNA methylation machinery in controlling inflammatory mRNA stability. These activities take place in various subcellular compartments and are differentially regulated during infection. In this way, mRNA-destabilising RBPs constitute a 'brake' on the immune system, which may ultimately be toggled therapeutically. I anticipate continued efforts in this area will lead to new methods of regaining control over inflammation in autoimmunity, selectively enhancing immunity in i

#Reranking

In [ ]:
from sentence_transformers import CrossEncoder

# Initialize the cross-encoder
device = "cuda" if torch.cuda.is_available() else "cpu"
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', device=device)

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
results = hybrid_search(col, query_dense_embedding, query_sparse_embedding, limit=3)
cross_encoder_input = [(question, context) for _, context in results]

#Generation through llama_index

In [ ]:
!pip install llama-index-llms-ollama

In [ ]:
! curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
import subprocess
process = subprocess.Popen("ollama serve", shell=True)
# credit: https://stackoverflow.com/questions/78437376/run-ollama-run-llama3-in-colab-raise-err-error-could-not-connect-to-ollama

# download the Mistral model
!ollama pull mistral

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠋ pulling manifest ⠋ pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏ 3.0 MB/4.1 GB                  pulling manifest 
pulling ff82381e

In [ ]:
from llama_index.llms.ollama import Ollama
# Initialize the Ollama LLM
llm = Ollama(model="mistral", request_timeout=300.0, temperature=0.1)

In [ ]:
from llama_index.llms.ollama import Ollama
from llama_index.core import (Settings, VectorStoreIndex, SimpleDirectoryReader, PromptTemplate)

# Define the prompt template for generating the answer
prompt_template = """
    You are a helpful AI assistant. Use the following pieces of information to answer the user's question. Provide a concise answer in one sentence only. If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Context: {context}
    Question: {question}
    """

In [ ]:
questions_data = data['question'].tolist()
contexts_data = data['contexts'].tolist()

In [ ]:
# Simulate the process for the first question
question = questions_data[0]

# Retrieve the pre-generated dense and sparse embeddings for the first question
embeddings = question_embeddings[0]  # Use the stored embeddings for the first question

query_dense_embedding = embeddings['dense'][0].tolist()  # Convert NumPy array to list of floats
query_sparse_embedding = embeddings['sparse']  # Use the sparse matrix in CSR format

# Perform the hybrid search using both embeddings
results = hybrid_search(col, query_dense_embedding, query_sparse_embedding, limit=3)

# Use the CrossEncoder to rerank the results
cross_encoder_input = [(question, context) for _, context in results]
scores = cross_encoder.predict(cross_encoder_input)

# Combine the results with the scores and sort by score
reranked_results = sorted(zip(results, scores), key=lambda x: x[1], reverse=True)

# Get the top context (best match)
top_context = reranked_results[0][0][1]  # The context of the top result

# Generate the answer using the Mistral model
prompt = f"Question: {question}\nContext: {top_context}\nAnswer:"
response = llm.complete(prompt)

# Display the generated answer
print(f"Generated Answer for question '{question}':\n{response}")

Generated Answer for question 'Which team scored the longest field goal kick of the game?':
 The team that kicked the longest field goal in this game was the Rams, with a kick made by Jeff Wilkins for 41 yards.


In [ ]:
# Initialize an empty list to store the generated answers
generated_answers = []

for i, question in enumerate(questions_data):
    # Retrieve the pre-generated dense and sparse embeddings for the current question
    embeddings = question_embeddings[i]  # Use the stored embeddings for the current question

    query_dense_embedding = embeddings['dense'][0].tolist()  # Convert NumPy array to list of floats
    query_sparse_embedding = embeddings['sparse']  # Use the sparse matrix in CSR format

    # Perform the hybrid search using both embeddings
    results = hybrid_search(col, query_dense_embedding, query_sparse_embedding, limit=3)

    # Use the CrossEncoder to rerank the results
    cross_encoder_input = [(question, context) for _, context in results]
    scores = cross_encoder.predict(cross_encoder_input)

    # Combine the results with the scores and sort by score
    reranked_results = sorted(zip(results, scores), key=lambda x: x[1], reverse=True)

    # Get the top context (best match)
    top_context = reranked_results[0][0][1]  # The context of the top result

    # Generate the answer using the Mistral model
    prompt = f"Question: {question}\nContext: {top_context}\nAnswer:"
    response = llm.complete(prompt)  # Generate the answer using the Mistral model

    # Append the generated answer to the list
    generated_answers.append(response)

    # Print the generated answer for each question
    print(f"Generated Answer for question '{question}':\n{response}")
    print('-' * 50)

Generated Answer for question 'Which team scored the longest field goal kick of the game?':
 The team that kicked the longest field goal in this game was the Rams, with a kick made by Jeff Wilkins for 41 yards.
--------------------------------------------------
Generated Answer for question 'How many percent were not  Irish?':
100% - 86.77% (Irish) - 22.5% (German) - 13.1% (Irish) - 9.8% (Italian) - 9.2% (English) - 8.1% ("American") - 6.0% (Polish) = 17.8% were not Irish or any of the other listed ancestries provided in the context. However, it's important to note that this calculation does not account for those who identified as Race (United States Census) or Race (United States Census) of any race, which was 1.91% of the population. To get a more accurate percentage of people who were not Irish, you would need to subtract these percentages from the total and then recalculate.
--------------------------------------------------
Generated Answer for question 'How many yards was the sec

In [ ]:
questions = data['question'].tolist()
contexts = data['contexts'].tolist()
ground_truths = data['ground_truth'].tolist()

In [ ]:
# Combine the result into a DataFrame
output_df = pd.DataFrame({
    'question': data['question'],
    'contexts': data['contexts'],
    'answer': generated_answers,    # Generated answers for each question
    'ground_truth': data['ground_truth']   # The ground truth from the original dataset
})
data['answer'] = generated_answers

In [ ]:
output_csv_path = 'new_result_halubench_short_Ollama-Mistral_llamaindex.csv'
data.to_csv(output_csv_path, index=False)

#Generation through langchain

In [ ]:
!pip install -q langchain ollama
!pip install -q langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 69.9 MB/s eta 0:00:00


In [ ]:
from langchain_community.llms import Ollama
langchain_llm = Ollama(
    model="mistral",
    temperature=0.1,
    num_ctx=8096,  # Context window size
    num_thread=8,  # Number of threads to use
    verbose=True,
    timeout=None
)

In [ ]:
from langchain.prompts import PromptTemplate

# Define the prompt template for generating the answer
prompt_template = """
You are a helpful AI assistant. Use the following pieces of information to answer the user's question.
Provide a concise answer in one sentence only. If you don't know the answer, just say that you don't know.
Context: {context}
Question: {question}
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])